In [1]:
import pandas as pd
import datetime as dt

pd.options.display.max_columns=1000
pd.options.display.max_rows = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%run configs.ipynb

In [2]:
# parameter cell
file_name = 'oct_21.csv'

In [3]:
# Parameters
file_name = "jan_21.csv"


# RRC Prod

## Ingest

In [4]:
production = pd.read_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', header = 5)

In [5]:
production.head()

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
0,NaN,Gas,05,164066.0,DUBOIS (BARNETT),MILES,1,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN
1,NaN,Gas,7B,211352.0,"GRANBURY, N.E. (ATOKA)",LANGDON,2,NaN,5422.0,4,0,NaN,NaN,4,0,NaN,NaN
2,NaN,Gas,7B,212409.0,"GRANBURY, N.E. (ATOKA)","JOHN J. MILES, JR. GAS UNIT",4,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN
3,NaN,Gas,7B,228695.0,"GRANBURY, N.E. (ATOKA)",LUCY UNIT,2,NaN,5699.0,12,0,NaN,NaN,12,0,NaN,NaN
4,NaN,Gas,7B,228926.0,"GRANBURY, N.E. (ATOKA)","MARSHALL, SAM A",2,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN


## Transform RRC

### Filter to District

In [6]:
#converting data field to string to handle varying schema interpreations 
#(certain months field will be interpreted as float and string depending sample rows pandas uses

production = production.loc[production['District'].astype(str).isin(['8.0', '08', '8'])]

In [7]:
production

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
7,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,119,36,NaN,NaN,155,0,NaN,NaN
8,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,58,0,NaN,NaN,58,0,NaN,NaN
9,Y,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,550,163,363,1.0,350,"2,422","2,422",2.0
10,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"2,221","10,443","9,088",0.0,"1,892","35,318","35,318",2.0
12,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,630,"1,695","1,594",0.0,441,"6,215","6,215",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,NaN,Gas,08,160203.0,WADDELL (GRAYBURG),"WADDELL, W.N.",310,NaN,8988.0,0,0,NaN,NaN,0,0,NaN,NaN
521,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,NaN,NaN,0,0,NaN,NaN
522,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,16,42,42,1.0,16,184,184,2.0
523,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,16,42,42,1.0,16,824,824,2.0


In [8]:
production.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 464 entries, 7 to 524
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Multiple Reports            47 non-null     object 
 1   Lease Type                  408 non-null    object 
 2   District                    464 non-null    object 
 3   RRC Identifier              464 non-null    float64
 4   Field Name                  464 non-null    object 
 5   Lease Name                  464 non-null    object 
 6   Gas Well ID                 236 non-null    object 
 7   Lease Total                 14 non-null     object 
 8   Commingle Permit No.        275 non-null    float64
 9   On Hand Beginning of Month  461 non-null    object 
 10  Production                  464 non-null    object 
 11  Volume                      208 non-null    object 
 12  Code                        208 non-null    float64
 13  On Hand End of Month        464 non

### Clean Oil Production Columns and Convert to Int

In [9]:
production['Volume'] = production.loc[:,'Volume'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))
production['Volume.1'] = production.loc[:,'Volume.1'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))

### Rename Volume to rrc_oil_volume

In [10]:
production.rename(columns = {'Volume': 'rrc_oil_volume'}, inplace = True)
production.rename(columns = {'Volume.1': 'rrc_gas_volume'}, inplace = True)

In [11]:
production['rrc_oil_volume'].sum()
production['rrc_gas_volume'].sum()

359609

1639292

### Normalize Field and Reservoir Names with Text Cleaning

In [12]:
production["Normalized_Field_Name"] = production['Field Name'].dropna()\
                                                            .apply(lambda x: str(x).split('(')[0]\
                                                                                    .split(',')[0]\
                                                                                    .strip())

In [13]:
production["Normalized_Reservoir_Name"] = production['Field Name'].dropna()\
                                        .apply(lambda x: str(x).split('(')[-1]\
                                                               .strip().replace(')',''))

### Get Date from Production File

In [14]:
def get_date_from_csv(filepath, col_number, row_number):
    str_date = pd.read_csv(filepath, usecols = [col_number]).iloc[row_number].values[0]
    return str_date.strip()

In [15]:
get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1)

'Jan 2021'

### Add Production Date to Production Data Frame

In [16]:
production.insert(0, 'Date', get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1))

In [17]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name
7,Jan 2021,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,119,36,0,NaN,155,0,0,NaN,ARMER,TUBB
8,Jan 2021,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,58,0,0,NaN,58,0,0,NaN,ARMER,6350
9,Jan 2021,Y,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,550,163,363,1.0,350,"2,422",2422,2.0,ARMER,6350
10,Jan 2021,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"2,221","10,443",9088,0.0,"1,892","35,318",35318,2.0,ARMER,6350
12,Jan 2021,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,630,"1,695",1594,0.0,441,"6,215",6215,2.0,ARMER,6350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Jan 2021,NaN,Gas,08,160203.0,WADDELL (GRAYBURG),"WADDELL, W.N.",310,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG
521,Jan 2021,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG
522,Jan 2021,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,16,42,42,1.0,16,184,184,2.0,WADDELL,GRAYBURG
523,Jan 2021,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,16,42,42,1.0,16,824,824,2.0,WADDELL,GRAYBURG


### Add columns for net production to Trust

In [18]:
production['Net Oil Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50
production['Net Oil Sales Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50 * 0.75

In [19]:
production['Net Gas Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50
production['Net Gas Sales Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50 * 0.75

In [20]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name,Net Oil Volume to Trust (RRC),Net Oil Sales Volume to Trust (RRC),Net Gas Volume to Trust (RRC),Net Gas Sales Volume to Trust (RRC)
7,Jan 2021,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,119,36,0,NaN,155,0,0,NaN,ARMER,TUBB,0.0,0.000,0.0,0.000
8,Jan 2021,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,58,0,0,NaN,58,0,0,NaN,ARMER,6350,0.0,0.000,0.0,0.000
9,Jan 2021,Y,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,550,163,363,1.0,350,"2,422",2422,2.0,ARMER,6350,181.5,136.125,1211.0,908.250
10,Jan 2021,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"2,221","10,443",9088,0.0,"1,892","35,318",35318,2.0,ARMER,6350,4544.0,3408.000,17659.0,13244.250
12,Jan 2021,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,630,"1,695",1594,0.0,441,"6,215",6215,2.0,ARMER,6350,797.0,597.750,3107.5,2330.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Jan 2021,NaN,Gas,08,160203.0,WADDELL (GRAYBURG),"WADDELL, W.N.",310,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG,0.0,0.000,0.0,0.000
521,Jan 2021,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG,0.0,0.000,0.0,0.000
522,Jan 2021,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,16,42,42,1.0,16,184,184,2.0,WADDELL,GRAYBURG,21.0,15.750,92.0,69.000
523,Jan 2021,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,16,42,42,1.0,16,824,824,2.0,WADDELL,GRAYBURG,21.0,15.750,412.0,309.000


### Write to File

In [21]:
production.to_csv(f'{processed_data_folder}/rrc_prod/{file_name}', index = False)

## Aggregated EDA

In [22]:
production.groupby(['Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'})\
                                            .sort_values(by = 'rrc_oil_volume', ascending= False)\
                                            .head(15)

,rrc_oil_volume
Normalized_Field_Name,
SAND HILLS,149474
MONAHANS,101681
ARMER,33985
DUNE,28064
UNIVERSITY WADDELL,11493
MARSTON RANCH,10753
KEYSTONE,9464
WADDELL,6299
CORDONA LAKE,1957


In [23]:
production.groupby(['Normalized_Reservoir_Name', 'Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'}).sort_values(by = 'rrc_oil_volume', ascending= False).sort_index()

rrc_oil_volume
Normalized_Reservoir_Name Normalized_Field_Name                
6350                      ARMER                           33985
7900                      EDWARDS -04-                        0
CLEAR FORK                LEA                               335
                          MONAHANS                            0
                          SAND HILLS                      29861
CLEAR FORK 4070           SAND HILLS                        151
CLEAR FORK, LOWER         MCKEE                               0
CLEAR FORK, MID.          MCKEE                               0
CLEARFORK                 MARSTON RANCH                   10753
                          MONAHANS                        95497
                          RM                                  0
COLBY                     KEYSTONE                         2140
CONSOLIDATED              KEYSTONE                         5704
CORDONA LAKE              CORDONA LAKE                        0
COWDEN, NORTH             COWDEN                            341
DEV.                      CORDONA LAKE                     1715
DEVONIAN                  CORDONA LAKE                      242
                          UNIVERSITY WADDELL              10673
                          WADDELL                           157
DUNE                      DUNE                            28064
ELLENBURGER               KERMIT                           1001
                          SAND HILLS                       1990
                          UNIVERSITY WADDELL                  0
GRAYBURG                  GOLDSMITH                           0
                          WADDELL                           416
HOLT                      RUNNING W                          33
                          SAND HILLS                          0
JUDKINS                   SAND HILLS                       5216
KERMIT                    KERMIT                            327
MCKEE                     BLOCK 27                            0
                          KERMIT                              0
                          RUNNING W                           0
MCKNIGHT                  SAND HILLS                      25124
MONAHANS, NORTH           MONAHANS                         6184
ORDOVICIAN                SAND HILLS                          0
PENN UPPER                BLOCK 31                          355
PENN.                     G. P. M.                          163
                          UNIVERSITY WADDELL                 30
QUEEN                     GOLDSMITH                           0
SAN ANDRES                C-BAR                               0
                          DELL                              153
                          KEYSTONE                         1620
                          LEA                                47
                          SHAFTER LAKE                     1039
SAN ANDRES CONS.          CRAWAR                              0
SAN ANDRES, CON.          GOLDSMITH                           0
SAN ANGELO                LEA                              1207
                          SAND HILLS                        119
SAN ANGELO, UPPER         SAND HILLS                      24984
TUBB                      -Y-                                 0
                          ARMER                               0
                          BLOCK B-21                          0
                          RUNNING W                          27
                          SAND HILLS                      29201
TUBB 4500                 CORDONA LAKE                        0
WADDELL                   RUNNING W                         221
                          WADDELL                          5726
WADDELL SD.               UNIVERSITY WADDELL                790
WICHITA-ALBANY            RUNNING W                        1190
                          SAND HILLS                      16185
WICHITA-ALBANY, GAS       SAND HILLS                        220
WOJCIK-MCELROY   